In [ ]:
#Mount Drive
from google.colab import drive
import os
drive.mount('/content/drive')

In [ ]:
!pip install tmdbv3api

In [ ]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13485 sha256=3c83bddbf0b5042ed62dbe9c02d548cfcc37f605da911d416f0dbbe218eb4f6e
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


#TMDb Clawer

In [ ]:
from tmdbv3api import TMDb,Movie,Discover,Genre,Keyword
import csv
import numpy as np
import pandas as pd

In [ ]:
API_V3_KEY = '55c9a195575d446d4c9271aa210bdd81'
tmdb = TMDb()
tmdb.api_key = API_V3_KEY
tmdb.language = 'en'
tmdb.debug = True
genres_list = {
    28:'Action',
    12:'Adventure',
    16:'Animation',
    35:'Comedy',
    80:'Crime',
    99:'Documentary',
    18:'Drama',
    14:'Fantasy',
    36:'History',
    27:'Horror',
    37:'Western',
    53:'Thriller',
    878:'Science Fiction',
    10751:'Family',
    10402:'Music',
    9648:'Mystery',
    10749:'Romance',
    10770:'TV Movie',
    10752:'War'
}

def get_genres_list():
  genre = Genre()
  genres = genre.movie_list()
  genres_list = {}
  for g in genres:
    genres_list[g['id']] = g['name']
  return genres_list

def get_movie_information(m):
  movie = Movie()
  info = {'tmdb_id' : None,
          'title' : None,
          'release_date' : None,
          'genres': None,
          'video_key':None,
          'overview':None,
          'trailer_url' : None,
          'keywords' : None,
          'popularity' : None,
          'vote_count' : None,
          'vote_average' : None
          }
  try:
    info['tmdb_id'] = m.id
    info['title'] = m.title
    info['release_date'] = m.release_date
    info['popularity'] = m.popularity
    info['vote_count'] = m.vote_count
    info['vote_average'] = m.vote_average
  except Exception as e:
    print('Cannot find movie id is :',m.id)
  try:
    genres = []
    ids = m.genre_ids
    for id in ids:
      genres.append(genres_list[id])
    info['genres'] = genres
  except Exception as e:
    print('Cannot find movie genres for :', m.id)
  try:
    movie_keywords = movie.keywords(m.id)['keywords']
    if movie_keywords is not None:
      keyws = []
      for keyw in movie_keywords:
        keyws.append(keyw['name'])
      info['keywords'] = keyws
    else:
      info['keywords'] = None
  except Exception as e:
    print('Cannot find keywords for :', m.id)
  try:
    info['overview'] = m.overview
  except Exception as e:
    info['overview'] = None
  try:
    trailer_keys = movie.videos(m.id)
    if trailer_keys is not None:
      for trailer_key in trailer_keys:
        if trailer_key['type'] == 'Trailer':
          if trailer_key['site'] == 'YouTube':
            info['trailer_url'] = 'https://www.youtube.com/watch?v='+ trailer_key['key']
            info['video_key'] = trailer_key['key']
            break
        else:
          info['trailer_url'] = None
    else:
      info['trailer_url'] = None
  except Exception as e:
    print('Can not find trailer url for :', m.id)
  return info

def discover_movies(year, num):
  movie_records =  []
  for i in range(int(num/20)):
    movie = Movie()
    discover = Discover()
    m_20 = discover.discover_movies({
        'primary_release_date.gte': '{y}-01-01'.format(y = year),
        'primary_release_date.lte': '{y}-12-31'.format(y = year),
        'page': '{page}'.format(page = i + 1),
        'sort_by':'vote_count.desc',
        }
    )
    for m in m_20:
      info = get_movie_information(m)
      movie_records.append(info.values())
  return movie_records

def save_movie_records(year,num):
  print('Writing file to csv')
  movie_records = discover_movies(year,num)
  with open('tmdb_movie_records_{y}_{n}.csv'.format(y = year,n = num),'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(meta)
  print('In {y}, total number:{n}'.format(y = year,n = num))

def movie_records(year_start,year_end,num):
  for i in range(year_start,year_end+1,1):
    save_movie_records(i,num)

In [ ]:
movie_records = discover_movies(2020,20)
meta_data = pd.DataFrame.from_records(movie_records,columns=['tmdb_id','title','release_date','genres','video_key','overview','trailer_url','keywords','popularity','vote_count','vote_average'])
meta_data

,tmdb_id,title,release_date,genres,video_key,overview,trailer_url,keywords,popularity,vote_count,vote_average
0,495764,Birds of Prey (and the Fantabulous Emancipatio...,2020-02-05,"[Action, Crime]",kGM4uYZzfu0,"Harley Quinn joins forces with a singer, an as...",https://www.youtube.com/watch?v=kGM4uYZzfu0,"[dc comics, based on comic, woman director, dc...",388.858,7510,7.1
1,454626,Sonic the Hedgehog,2020-02-12,"[Action, Science Fiction, Comedy, Family]",szby7ZHLnkA,Based on the global blockbuster videogame fran...,https://www.youtube.com/watch?v=szby7ZHLnkA,"[video game, san francisco, california, friend...",99.571,6844,7.5
2,38700,Bad Boys for Life,2020-01-15,"[Thriller, Action, Crime]",jKCj3XuPG8M,Marcus and Mike are forced to confront new thr...,https://www.youtube.com/watch?v=jKCj3XuPG8M,"[detective, police, sequel, police officer, po...",324.430,6296,7.2
3,508442,Soul,2020-12-25,"[Family, Animation, Comedy, Drama, Music, Fant...",xOsLIiBStEs,Joe Gardner is a middle school teacher with a ...,https://www.youtube.com/watch?v=xOsLIiBStEs,"[new york city, jazz, self-discovery, jazz sin...",493.296,6135,8.3
4,664413,365 Days,2020-02-07,"[Drama, Romance]",J79D3hny-e8,"A woman falls victim to a dominant mafia boss,...",https://www.youtube.com/watch?v=J79D3hny-e8,"[kidnapping, nightclub, gangster, yacht, mansi...",28.883,5773,7.1
5,577922,Tenet,2020-08-22,"[Action, Thriller, Science Fiction]",LdOM0x0XDMo,Armed with only one word - Tenet - and fightin...,https://www.youtube.com/watch?v=LdOM0x0XDMo,"[spy, assassin, mumbai (bombay), india, time t...",270.617,5346,7.3
6,464052,Wonder Woman 1984,2020-12-16,"[Fantasy, Action, Adventure]",sfM7_JLk-84,A botched store robbery places Wonder Woman in...,https://www.youtube.com/watch?v=sfM7_JLk-84,"[dc comics, superhero, based on comic, 1980s, ...",531.763,5210,6.7
7,337401,Mulan,2020-09-04,"[Adventure, Fantasy]",KK8FHdFluOQ,When the Emperor of China issues a decree that...,https://www.youtube.com/watch?v=KK8FHdFluOQ,"[china, martial arts, hero, false identity, ho...",238.260,4795,7.0
8,508439,Onward,2020-02-29,"[Animation, Family, Adventure, Comedy, Fantasy]",gn5QmllRCn4,"In a suburban fantasy world, two teenage elf b...",https://www.youtube.com/watch?v=gn5QmllRCn4,"[elves, magic, dead father, dead parent, fanta...",58.280,4277,7.8
9,583083,The Kissing Booth 2,2020-07-24,"[Comedy, Romance]",fjVonI2oVeM,"With college decisions looming, Elle juggles h...",https://www.youtube.com/watch?v=fjVonI2oVeM,[],56.207,4054,7.9


#IMDb to TMDb

In [ ]:
import json
import requests
from fake_useragent import FakeUserAgentError, UserAgent
from tmdbv3api import TMDb,Movie,Discover,Genre,Keyword
import csv
import numpy as np
import pandas as pd

In [ ]:
API_V3_KEY = '55c9a195575d446d4c9271aa210bdd81'
tmdb = TMDb()
tmdb.api_key = API_V3_KEY
tmdb.language = 'en'
tmdb.debug = True

In [ ]:
def imdb_id_2_tmdb_id(imdb_id):
  tmdb_url = 'https://api.themoviedb.org/3/'
  api_key = API_V3_KEY
  try:
    ua = UserAgent()
    header = {'User-Agent': str(ua.chrome)}
  except FakeUserAgentError:
    header = {'User-Agent': ''}
  url = tmdb_url + 'find/{}'.format(mid)
  params = {
            'api_key': api_key,
            'language': 'en-US',
            'external_source': 'imdb_id'
            }
  tmdb_id = None
  try:
    r = requests.get(url, params=params, headers=header)
    data = json.loads(r.text)
    if data.get('status_code'):
      print('ERROR! status code: {}.'.format(data['status_code']))
    movie_results = data['movie_results']
    if len(movie_results) != 1:
      print('{} search results: {}'.format(mid, len(movie_results)))
    else:
      tmdb_id = '{}'.format(movie_results[0]['id'])
  except Exception as e:
    print('{} {}'.format(tmdb_id, e))
  return tmdb_id

def imdb2tmdb(imdb_id):
  url_prefix = 'https://api.themoviedb.org/3/'
  try:
    api_key = API_V3_KEY
  except Exception:
    print('No apikey found')
  try:
    ua = UserAgent()
    header = {'User-Agent': str(ua.chrome)}
  except FakeUserAgentError:
    header = {'User-Agent': ''}
  url = url_prefix + 'find/{}'.format(imdb_id)
  params = {
  'api_key': api_key,
  'language': 'en-US',
  'external_source': 'imdb_id'
  }
  try:
    r = requests.get(url, params=params, headers=header)
    data = json.loads(r.text)
    movie_results = data['movie_results']
    tmdb_id = movie_results[0]['id']
    movie = Movie()
    m = movie.details(tmdb_id)
    info = get_movie_information(m,imdb_id)
  except Exception:
    print('can not find movie for:',imdb_id)
    info = None
    
  return info

def get_movie_information(m,imdb_id):
  movie = Movie()
  info = {
          'imdb_id' : imdb_id,
          'tmdb_id' : None,
          'title' : None,
          'release_date' : None,
          'genres': None,
          'video_key':None,
          'overview':None,
          'trailer_url' : None,
          'keywords' : None,
          'popularity' : None,
          'vote_count' : None,
          'vote_average' : None
          }
  try:
    info['tmdb_id'] = m.id
    info['title'] = m.title
    info['release_date'] = m.release_date
    info['popularity'] = m.popularity
    info['vote_count'] = m.vote_count
    info['vote_average'] = m.vote_average
  except Exception as e:
    print('Cannot find movie id is :',m.id)
  try:
    genres = []
    for i in m.genres:
      genres.append(i['name'])
    info['genres'] = genres
  except Exception as e:
    print('Cannot find movie genres for :', m.id)

  try:
    movie_keywords = movie.keywords(m.id)['keywords']
    if movie_keywords is not None:
      keyws = []
      for keyw in movie_keywords:
        keyws.append(keyw['name'])
      info['keywords'] = keyws
    else:
      info['keywords'] = None

  except Exception as e:
    print('Cannot find keywords for :', m.id)
  try:
    info['overview'] = m.overview
  except Exception as e:
    info['overview'] = None

  try:
    trailer_keys = movie.videos(m.id)
    if trailer_keys is not None:
      for trailer_key in trailer_keys:
        if trailer_key['type'] == 'Trailer':
          if trailer_key['site'] == 'YouTube':
            info['trailer_url'] = 'https://www.youtube.com/watch?v='+ trailer_key['key']
            info['video_key'] = trailer_key['key']
            break
        else:
          info['trailer_url'] = None
    else:
      info['trailer_url'] = None
  except Exception as e:
    print('Can not find trailer url for :', m.id)
  return info

In [ ]:
id_path = '../data/86_movie/MovieNet_1K/list.v1.txt'
ids = pd.read_csv(id_path, header = None)
ids

,0
0,tt0032138
1,tt0035423
2,tt0038650
3,tt0045537
4,tt0047396
...,...
1095,tt7055592
1096,tt7131870
1097,tt7160070
1098,tt7180392


In [ ]:
meta_data = imdb2tmdb(ids[0][10])
meta_data

{'genres': ['Drama', 'History'],
 'imdb_id': 'tt0049833',
 'keywords': ['moses',
  'israel',
  'egypt',
  'miracle',
  'slavery',
  'ten commandments',
  'bible',
  'epic',
  'religion',
  'judaism',
  'ancient egypt',
  'old testament',
  'pharaoh',
  'passover',
  '13th century bc'],
 'overview': 'Escaping death, a Hebrew infant is raised in a royal household to become a prince. Upon discovery of his true heritage, Moses embarks on a personal quest to reclaim his destiny as the leader and liberator of the Hebrew people.',
 'popularity': 37.434,
 'release_date': '1956-10-05',
 'title': 'The Ten Commandments',
 'tmdb_id': 6844,
 'trailer_url': 'https://www.youtube.com/watch?v=EiLmKxiTT3g',
 'video_key': 'EiLmKxiTT3g',
 'vote_average': 7.7,
 'vote_count': 985}

#IMDb Clawer

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import FakeUserAgentError, UserAgent
from termcolor import colored

In [ ]:
def parse_imdb_page(imdb_id):
        url_prefix = 'https://www.imdb.com/title'
        try:
          ua = UserAgent()
          header = {'User-Agent':str(ua.chrome)}
        except FakeUserAgentError:
          header = {'User-Agent':''}
        try:
            response = requests.get(
                '{}/{}'.format(url_prefix, imdb_id), headers=header)
            content = response.text
        except Exception as e:
            print(e)
        page_soup = BeautifulSoup(content, 'lxml')
        info = {
            'imdb_id': imdb_id,
            'title': None,
            'genres': None,
            'country': None,
            'version': [],
            'storyline': None
        }
        try:
            title_meta = page_soup.find('meta', property='og:title')
            title = title_meta['content']
            info['title'] = title.split('-')[0].strip()
        except Exception as e:
            print('Cannot find title. {}: {}'.format(imdb_id, e))
        try:
            titleStoryLine = page_soup.find('div', {'id': 'titleStoryLine'})
            divs = titleStoryLine.find_all('div', class_='canwrap')
            for div in divs:
                p = div.find('p')
                if p is not None:
                    story = p.find('span').get_text()
                    info['storyline'] = story.strip()
                else:
                    title = div.find('h4')
                    if title is not None:
                        title = title.string
                        if title.find('Genres') >= 0:
                            genres = []
                            genres_as = div.find_all('a')
                            for genres_a in genres_as:
                                genres.append(genres_a.string.strip())
                            info['genres'] = genres
        except Exception as e:
            print(
                colored(
                    'Cannot find storyline or genres. {}: {}'.format(imdb_id, e),
                    'red'))
        try:
            titleStoryLine = page_soup.find('div', {'id': 'titleDetails'})
            divs = titleStoryLine.find_all('div', class_='txt-block')
            for div in divs:
                title = div.find('h4')
                if title is not None:
                    title = title.string
                    if title.find('Country') >= 0:
                        country_a = div.find('a')
                        info['country'] = country_a.string.strip()
                    if title.find('Runtime') >= 0:
                        text = div.get_text().strip().replace('Runtime:', '')
                        runtimes = text.split('|')
                        for runtime in runtimes:
                            minutes = runtime.split('min')[0].strip()
                            description = runtime.split('min')[1].strip()
                            description = description.replace('(', '')
                            description = description.replace(')', '')
                            info['version'].append({
                                'runtime': minutes + ' min',
                                'description': description
                            })
        except Exception as e:
            print(
                colored(
                    'Cannot find country or runtime. {}: {}'.format(mid, e),
                    'red'))
        return info

def parse_credits(imdb_id):
        info = {
            'imdb_id': imdb_id,
            'cast': None,
            'director': None,
        }
        try:
            response = requests.get(
                '{}/{}/{}'.format(url_prefix, imdb_id, 'fullcredits'),
                headers=header)
            content = response.text
            page_soup = BeautifulSoup(content, 'lxml')
            credit_div = page_soup.find('div', {'id': 'fullcredits_content'})
            d_table = credit_div.find('table', class_='simpleCreditsTable')
            td = d_table.find('td', class_='name')
            tda = td.find('a')
            pid = tda['href'].split('/')[2]
            name = tda.string.strip()
            info['director'] = {'id': pid, 'name': name}
            cast_table = credit_div.find('table', class_='cast_list')
            if cast_table is not None:
                trs = cast_table.find_all('tr')
                cast = []
                for tr in trs:
                    td = tr.find('td', class_='primary_photo')
                    if td is None:
                        continue
                    tda = td.find('a')
                    pid = tda['href'].split('/')[2].strip()
                    name = tda.find('img')['title'].strip()
                    td = tr.find('td', class_='character')
                    character = td.get_text().strip()
                    character = character.replace('\n', '')
                    if character.find('uncredited') >= 0:
                        continue
                    cast.append({
                        'id': pid,
                        'name': name,
                        'character': character
                    })
                info['cast'] = cast
        except Exception as e:
            print(colored('Cannot find credits. {}: {}'.format(mid, e), 'red'))
        return info

def parse_storyline(imdb_id):
        info = {'storyline': None}
        try:
            response = requests.get(
                '{}/{}/plotsummary'.format(url_prefix, imdb_id),
                headers=header)
            content = response.text
            page_soup = BeautifulSoup(content, 'lxml')
            ul = page_soup.find('ul', {'id': 'plot-synopsis-content'})
            storyline = ul.get_text().strip()
            if storyline.find('Edit page') >= 0:
                storyline = None
            info['storyline'] = storyline
        except Exception as e:
            print(colored('{} {}'.format(imdb_id, e), 'red'))
        return info

In [ ]:
id_path = '../data/86_movie/MovieNet_1K/list.v1.txt'
ids = pd.read_csv(id_path, header = None)
ids

,0
0,tt0032138
1,tt0035423
2,tt0038650
3,tt0045537
4,tt0047396
...,...
1095,tt7055592
1096,tt7131870
1097,tt7160070
1098,tt7180392


In [ ]:
info = parse_imdb_page(ids[0][10])
info

{'country': 'USA',
 'genres': ['Adventure', 'Drama'],
 'imdb_id': 'tt0049833',
 'storyline': 'To escape the edict of Egypt\'s Pharaoh Rameses I (Ian Keith), condemning all newborn Hebrew males, the infant Moses (Fraser C. Heston) is set adrift on the Nile in a reed basket. Saved by the pharaoh\'s daughter Bithiah (Nina Foch), he is adopted by her and brought up in the court of her brother, Pharaoh Sethi (Sir Cedric Hardwicke). Moses (Charlton Heston) gains Sethi\'s favor and the love of the throne Princess Nefretiri (Anne Baxter), as well as the hatred of Sethi\'s son, Rameses II (Yul Brynner). When his Hebrew heritage is revealed, Moses is cast out of Egypt, and makes his way across the desert where he marries, has a son, and is commanded by God to return to Egypt to free the Hebrews from slavery. In Egypt, Moses\' fiercest enemy proves to be not Rameses II, but someone near to him who can "harden his heart".',
 'title': 'The Ten Commandments (1956)',
 'version': [{'description': '', 

In [ ]:
info = parse_credits(ids[0][10])
info

{'cast': [{'character': 'Moses', 'id': 'nm0000032', 'name': 'Charlton Heston'},
  {'character': 'Rameses', 'id': 'nm0000989', 'name': 'Yul Brynner'},
  {'character': 'Nefretiri', 'id': 'nm0000879', 'name': 'Anne Baxter'},
  {'character': 'Dathan', 'id': 'nm0000064', 'name': 'Edward G. Robinson'},
  {'character': 'Sephora', 'id': 'nm0001119', 'name': 'Yvonne De Carlo'},
  {'character': 'Lilia', 'id': 'nm0656428', 'name': 'Debra Paget'},
  {'character': 'Joshua', 'id': 'nm0001135', 'name': 'John Derek'},
  {'character': 'Sethi       (as Sir Cedric Hardwicke)',
   'id': 'nm0362567',
   'name': 'Cedric Hardwicke'},
  {'character': 'Bithiah', 'id': 'nm0001225', 'name': 'Nina Foch'},
  {'character': 'Yochabel', 'id': 'nm0779549', 'name': 'Martha Scott'},
  {'character': 'Memnet', 'id': 'nm0000752', 'name': 'Judith Anderson'},
  {'character': 'Baka', 'id': 'nm0001637', 'name': 'Vincent Price'},
  {'character': 'Aaron', 'id': 'nm0001017', 'name': 'John Carradine'},
  {'character': 'Miriam', 'i

In [ ]:
info = parse_storyline(ids[0][10])
info

{'storyline': 'The film covers the life of Moses from his discovery in a basket floating on the Nile as a baby by Bithiah, a childless young widow and daughter of the then-Pharaoh, Rameses I, to his prohibition from entering the land of Israel in the wake of God\'s judgment on him at the waters of Meribah. In between, the film depicts the early adulthood of Moses as a beloved foster son of Pharaoh Seti I (brother of Bithiah) and general of his armies, his romance with Throne Princess Nefertari (or Nefretiri, as she is called in the film) and rivalry with the Pharaoh\'s own son, Prince Rameses II.Shortly after Moses\' birth, Rameses I had ordered the slaying of all firstborn male Hebrews to prevent the prophecy of the Deliverer from coming true. Moses\' mother (called "Yoshebel" in the film) had set him adrift on the Nile to escape, with his sister Miriam watching from a hidden spot. Bithiah discovers the Ark while playing with other young women in the banks of the Nile. She orders them